# Segmentation to btrack BATCH PROCESS

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [31]:
import glob
import os
import btrack
print('Btrack version', btrack.__version__)
import napari
print('Napari version', napari.__version__)
import numpy as np

from skimage.io import imread
from natsort import natsorted
from napari_animation import AnimationWidget
from tqdm import tqdm
import tifffile as tiff
from datetime import datetime

### function to load images efficiently but presents problems when loading many arrays sequentially i.e masks and raw image data for regionprops
def stack_generator(files):
    """Stack generator"""
    
    for filename in tqdm(files):
        img = imread(filename)
        yield img
        
### function to load images as numpy array, slower but works better with regionprops
def image_arr(files):
    """Image stack as as numpy array."""
    
    stack = []
    for filename in tqdm(files):
        img = imread(filename)
        stack.append(img)
    return np.stack(stack, axis=0)

Btrack version 0.4.1
Napari version 0.4.7


# Set root folder and find expt IDs

In [5]:
root_folder = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/'
expt_IDs = natsorted([ID for ID in os.listdir(root_folder) if 'MK' in ID])
expt = expt_IDs[0]
pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])

# Run batch tracking job

In [ ]:
for pos in tqdm(pos_list):
    try:

        ### display info
        print(pos, 'starting')

        print('Finding mask files', pos)
        ### find masks and sort numerically
        mask_dir = os.path.join(root_folder,'{}/{}/{}_cp_masks'.format(expt, pos, pos))
        files = natsorted(glob.glob(os.path.join(mask_dir, '*.png')))
        #print(files)
        if len(files) == 0:
            print('No masks found for', pos, ', continuing to next position')
            continue

        print('Filtering mask files', pos)
        ## filter files
        gfp_files = [file for file in files if 'channel001' in file]
        rfp_files = [file for file in files if 'channel002' in file]

        print('Loading mask files', pos)
        ### load images
        gfp_generator = stack_generator(gfp_files)
        rfp_generator = stack_generator(rfp_files)

        print('Localising objects', pos)
        ### localise objects in images
        gfp_obj_from_generator = btrack.utils.segmentation_to_objects(gfp_generator, properties = ('area', 'major_axis_length'))
        rfp_obj_from_generator = btrack.utils.segmentation_to_objects(rfp_generator, properties = ('area', 'major_axis_length'))

        print('Tracking GFP objects', pos)
        ### track gfp objects
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:

            # configure the tracker using a config file
            tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json')
            tracker.max_search_radius = 10

            # append the objects to be tracked
            #tracker.append(gfp_obj_from_arr) #obj_from_generator)
            tracker.append(gfp_obj_from_generator)

            # set the volume
            tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

            # track them (in interactive mode)
            tracker.track_interactive(step_size=100)

            # generate hypotheses and run the global optimizer
            tracker.optimize()
            
            ## export with guaranteed unique fn 
            date = datetime.now().strftime("%Y_%m_%d_%I_%M_%S_%p")
            tracker.export(os.path.join(root_folder, '{}/{}'.format(expt, pos), 'gfp_tracks_{}.h5'.format(date)), obj_type='obj_type_1')

            # get the tracks in a format for napari visualization
            #gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
            #gfp_tracks = tracker.tracks

        print('Tracking RFP objects', pos)
        ### track rfp objects 
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:

            # configure the tracker using a config file
            tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_scribble_sparse.json')
            tracker.max_search_radius = 10

            # append the objects to be tracked
            #tracker.append(rfp_obj_from_arr) #obj_from_generator)
            tracker.append(rfp_obj_from_generator)

            # set the volume
            tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

            # track them (in interactive mode)
            tracker.track_interactive(step_size=100)

            # generate hypotheses and run the global optimizer
            tracker.optimize()
            
            ## export with guaranteed unique fn 
            date = datetime.now().strftime("%Y_%m_%d_%I_%M_%S_%p")
            tracker.export(os.path.join(root_folder, '{}/{}'.format(expt, pos), 'rfp_tracks_{}.h5'.format(date)), obj_type='obj_type_2')

            # get the tracks in a format for napari visualization
            #rfp_data, rfp_properties, rfp_graph = tracker.to_napari(ndim=2)
            #rfp_tracks = tracker.tracks
        print('Finished', pos)
        
    except:
        print(pos, 'Failed to generate tracking file (masks probably missing)')

  0%|          | 0/24 [00:00<?, ?it/s]

Pos0 starting
Finding mask files Pos0


[INFO][2021/05/21 05:11:04 PM] Localizing objects from segmentation...


Filtering mask files Pos0
Loading mask files Pos0
Localising objects Pos0


[INFO][2021/05/21 05:15:02 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/21 05:15:06 PM] ...Found 563356 objects in 1014 frames.
[INFO][2021/05/21 05:15:06 PM] Localizing objects from segmentation...
[INFO][2021/05/21 05:15:28 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/21 05:15:28 PM] ...Found 377 objects in 419 frames.
[INFO][2021/05/21 05:15:28 PM] Loaded btrack: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2021/05/21 05:15:28 PM] btrack (v0.4.1) library imported
[INFO][2021/05/21 05:15:28 PM] Setting max XYZ search radius to: 100
[INFO][2021/05/21 05:15:28 PM] Starting BayesianTracker session
[INFO][2021/05/21 05:15:28 PM] Loading configuration file: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json
[INFO][2021/05/21 05:15:28 PM] Loading motion model: b'MDCK_motion_Kristina'
[INFO][2021/05/21 05:15:28 PM] Setting max XYZ search radius to: 10
[INFO][2021/05/21 05:15:28 PM] Objects are

...

Tracking RFP objects Pos0


[INFO][2021/05/21 05:18:22 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos0/rfp_tracks_2021_05_21_05_18_22_PM.h5
[INFO][2021/05/21 05:18:22 PM] Ending BayesianTracker session
  4%|▍         | 1/24 [07:19<2:48:18, 439.07s/it]

Finished Pos0
Pos1 starting
Finding mask files Pos1
No masks found for Pos1 , continuing to next position
Pos2 starting
Finding mask files Pos2
No masks found for Pos2 , continuing to next position
Pos3 starting
Finding mask files Pos3
No masks found for Pos3 , continuing to next position
Pos4 starting
Finding mask files Pos4
No masks found for Pos4 , continuing to next position
Pos5 starting
Finding mask files Pos5


[INFO][2021/05/21 05:18:23 PM] Localizing objects from segmentation...


Filtering mask files Pos5
Loading mask files Pos5
Localising objects Pos5


# check progress

In [1]:
from hurry.filesize import size

In [9]:
### number of supposed files
print('positions with tracking files')
track_file_dict = {}
for pos in pos_list:
        #track_file_dict[pos] = {}
        ### display info
        #print(pos, 'starting')

        #print('Finding mask files', pos)
        ### find masks and sort numerically
        mask_dir = os.path.join(root_folder,'{}/{}/{}_cp_masks'.format(expt, pos, pos))
        files = natsorted(glob.glob(os.path.join(mask_dir, '*.png')))
        #print(pos, len(files))
        if len(files) > 0:
            print(pos, 'should have tracking file')
        #print(len(files), pos)
### number of actual files
        track_folder = os.path.join(root_folder,expt, pos)
        files = natsorted(glob.glob(os.path.join(track_folder, '*.h5')))
        if len(files) > 0:
            #print(pos, 'has the following tracks', [(os.path.basename(fn),os.path.getsize(fn)) for fn in files])
            #track_file_dict[pos] = [(os.path.basename(fn),size(os.path.getsize(fn))) for fn in files]
            track_file_dict[pos] = {}
            for fn in files:
                track_file_dict[pos][os.path.basename(fn)] = size(os.path.getsize(fn))

positions with tracking files
Pos0 should have tracking file
Pos5 should have tracking file
Pos6 should have tracking file
Pos7 should have tracking file
Pos10 should have tracking file
Pos11 should have tracking file
Pos12 should have tracking file
Pos15 should have tracking file
Pos16 should have tracking file
Pos17 should have tracking file
Pos20 should have tracking file
Pos21 should have tracking file
Pos22 should have tracking file


In [7]:
track_file_dict

{'Pos0': {'gfp_tracks_2021_05_21_05_18_15_PM.h5': '20M',
  'gfp_tracks_2021_05_21_05_29_08_PM.h5': '20M'},
 'Pos5': {'gfp_tracks.h5': '6M',
  'gfp_tracks_2021_05_21_05_21_31_PM.h5': '6M',
  'gfp_tracks_2021_05_21_05_32_08_PM.h5': '6M',
  'rfp_tracks.h5': '22K',
  'rfp_tracks_2021_05_21_05_21_33_PM.h5': '22K',
  'rfp_tracks_2021_05_21_05_32_11_PM.h5': '22K'},
 'Pos6': {'gfp_tracks.h5': '7M',
  'gfp_tracks_2021_05_21_05_35_20_PM.h5': '7M',
  'rfp_tracks.h5': '15K',
  'rfp_tracks_2021_05_21_05_35_23_PM.h5': '15K'},
 'Pos7': {'gfp_tracks.h5': '7M',
  'gfp_tracks_2021_05_21_05_38_33_PM.h5': '7M',
  'rfp_tracks.h5': '29K',
  'rfp_tracks_2021_05_21_05_38_36_PM.h5': '29K'},
 'Pos10': {'gfp_tracks_2021_05_21_05_42_19_PM.h5': '8M',
  'rfp_tracks_2021_05_21_05_42_23_PM.h5': '58K'},
 'Pos11': {'gfp_tracks_2021_05_21_05_45_15_PM.h5': '5M',
  'rfp_tracks_2021_05_21_05_45_17_PM.h5': '62K'},
 'Pos12': {'gfp_tracks_2021_05_21_05_49_29_PM.h5': '11M',
  'rfp_tracks_2021_05_21_05_49_34_PM.h5': '182K'},
 '

# and generate glimpses

(borrowed from other notebook to run sequentially as i want to go to the pub now instead of waiting for this to finish before running glimpse creator ty v much)

In [ ]:
import btrack
import matplotlib.pyplot as plt
import os
from skimage.io import imread, imsave, imshow
from tqdm import tqdm
import numpy as np
from pathlib import Path
from natsort import natsorted
import warnings
warnings.filterwarnings('ignore') ###bc imsave throws up low contrast warnings

In [ ]:
expt_root_folder = root_folder

channels = ['003'] ### bf 000 gfp 001 rfp 002 irfp 003

focal_time_range = (0, 800) ### this particular expt stays in focus in this time range

for pos in pos_list:
    print('starting', pos)
    tracks_path = os.path.join(expt_root_folder,'{}/{}/gfp_tracks.h5'.format(expt, pos))

    with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
        wt_tracks = hdf.tracks

    ### filter tracks that should contain a full cell cycle in (1 timepoint/frame = 4min, cc time approx 200 frames based on quick look)
    long_tracks = [track for track in wt_tracks if len(track) > 200]

    ### filter tracks that spend a lot of time out of focus
    tracks = [track for track in long_tracks if track.t[-1] < focal_time_range[1]]

    ### show number of filtered tracks
    print('number of glimpses to be generated:', len(tracks))

    ### find images and set output
    image_folder = os.path.join(expt_root_folder,expt, pos, pos+'_stacks')
    output_dir = '/home/nathan/data/kraken/pcna/glimpses/{}/{}'.format(expt, pos)

    for channel in channels:
        ### load images
        print('Loading image channel', channel)
        images = imread(os.path.join(image_folder, 'channel{}.tif'.format(channel)))
        print('Image loaded')
        ### create output directory
        output_ch_dir = os.path.join(output_dir, 'channel'+channel)
        Path(output_ch_dir).mkdir(parents=True, exist_ok=True)
        #os.mkdir(output_ch_dir) if os.path.exists(output_ch_dir) == False else print(output_ch_dir, 'already exists')

        for cell in tqdm(tracks):
            try:
                ### get length of track/glimpse 
                length = range(cell.t[0], cell.t[-1])
                ### get the xy coords of track/glimpse 
                y, x = cell.x, cell.y #### WHY IS THE COORDINATES TRANSPOSED???? 

                ### iterate over all frames cropping
                for j, i in enumerate(length): ## need to have something that solves skipped frames # not including the last frame bc list index out of range ie there is one more i than there
                    ### obtain correct frame
                    image = images[i]
                    ### obtain coordinates of bounding box of glimpse (centered on cell)
                    x1, x2, y1, y2 = x[j], x[j]+200, y[j], y[j]+200 #coordinates to crop by, shifted bearing in mind a further coord shift (+100,+100) occurs due to image padding            
                    ### pad image incase cell goes off FOV
                    image = np.pad(image, 100, 'constant', constant_values = 0) 
                    ## glimpse creation by cropping original image
                    glimpse = image[int(x1): int(x2), int(y1): int(y2)]
                    ### fraction of track this frame is at
                    age = round(i/cell.t[-1],3) 
                    fn = 'cell_ID_' + str(cell.ID) + '_channel' + channel + '_t{}_age{}.tif'.format(i, age)
                    ### glimpse output
                    imsave(os.path.join(output_ch_dir, fn), glimpse) 
                print('cell ID',cell.ID, 'glimpse saved')

            except:
                print(cell.ID, 'failed to produce glimpse')


# Generate tracking files with iRFP information in

##### Set root folder and find expt IDs

In [5]:
root_folder = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/'
expt_IDs = natsorted([ID for ID in os.listdir(root_folder) if 'MK' in ID])
expt_IDs
#pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])

['MK0000', 'MK0001']

In [80]:
log = []
for expt in expt_IDs:
    message = 'Starting experiment' + str(expt)
    log.append(message)
    print(message)
    pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])
    for pos in tqdm(pos_list):
        try:
                ### display info
                message = str(pos), 'starting'
                log.append(message)
                print(message)
                message = 'Finding mask files'+str(pos)
                log.append(message)
                print(message)

                ### find masks and sort numerically
                mask_dir = os.path.join(root_folder,'{}/{}/{}_cp_masks'.format(expt, pos, pos))
                files = natsorted(glob.glob(os.path.join(mask_dir, '*.png')))
                #print(files)
                if len(files) == 0:
                    message = 'No masks found for'+ str(pos)+ ', continuing to next position'
                    log.append(message)
                    print(message)
                    continue

                ## filter files
                message = 'Filtering mask files'+ str(pos)
                log.append(message)
                print(message)
                gfp_files = [file for file in files if 'channel001' in file]
                rfp_files = [file for file in files if 'channel002' in file]

                ### find actual images of cells to extract feature information
                message = 'Finding corresponding raw image files'+ str(pos)
                log.append(message)
                print(message)
                raw_image_dir = os.path.join(root_folder,'{}/{}/{}_raw'.format(expt, pos, pos))
                irfp_files = natsorted(glob.glob(os.path.join(raw_image_dir, '*channel003*.tif')))

                ### load all images and localise objs
                message = 'Loading all images'+ str(pos)
                log.append(message)
                print(message)

                ### old method used generator
    #             gfp_generator = stack_generator(gfp_files)
    #             rfp_generator = stack_generator(rfp_files)
    #             irfp_generator = stack_generator(irfp_files)

                ### new method uses stacks
                gfp_stack = image_arr(gfp_files)
                rfp_stack = image_arr(rfp_files)
                irfp_stack = image_arr(irfp_files)

                ### localise objects in images
                message = 'Localising objects'+str(pos)
                log.append(message)
                print(message)

                ### old method using generator
    #             gfp_obj_from_generator = btrack.utils.segmentation_to_objects(gfp_generator, properties = ('area', 'major_axis_length'))
    #             rfp_obj_from_generator = btrack.utils.segmentation_to_objects(rfp_generator, properties = ('area', 'major_axis_length'))
    #             irfp_obj_from_generator = btrack.utils.segmentation_to_objects(gfp_generator, irfp_generator,properties = ('area','major_axis_length', 'mean_intensity'))

                ### new method using stacks
                gfp_obj_from_arr = btrack.utils.segmentation_to_objects(gfp_stack, properties=('area','major_axis_length'))#, scale=(2., 1.))
                rfp_obj_from_arr = btrack.utils.segmentation_to_objects(rfp_stack, properties=('area', 'major_axis_length'))#, scale=(2., 1.))
                irfp_obj_from_arr = btrack.utils.segmentation_to_objects(gfp_stack, irfp_stack, properties=('area','major_axis_length', 'mean_intensity'))#, scale=(2., 1.))

                ### track irfp objects
                message = 'Tracking iRFP objects'+ str(pos)
                log.append(message)
                print(message)                             
                # initialise a tracker session using a context manager
                with btrack.BayesianTracker() as tracker:

                    # configure the tracker using a config file
                    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json')
                    tracker.max_search_radius = 10

                    # append the objects to be tracked
                    tracker.append(irfp_obj_from_arr)
                    #tracker.append(irfp_obj_from_generator)

                    # set the volume
                    tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

                    # track them (in interactive mode)
                    tracker.track_interactive(step_size=100)

                    # generate hypotheses and run the global optimizer
                    tracker.optimize()

                    ## export with guaranteed unique fn 
                    date = datetime.now().strftime("%Y_%m_%d_%I_%M_%S_%p")
                    tracker.export(os.path.join(root_folder, '{}/{}'.format(expt, pos), 'irfp_tracks_{}.h5'.format(date)), obj_type='obj_type_3')

                    # get the tracks in a format for napari visualization
                    #gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
                    #gfp_tracks = tracker.tracks

                ### track gfp objects
                message = 'Tracking GFP objects'+ str(pos)
                log.append(message)
                print(message)                             
                # initialise a tracker session using a context manager
                with btrack.BayesianTracker() as tracker:

                    # configure the tracker using a config file
                    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json')
                    tracker.max_search_radius = 10

                    # append the objects to be tracked
                    tracker.append(gfp_obj_from_arr)
                    #tracker.append(gfp_obj_from_generator)

                    # set the volume
                    tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

                    # track them (in interactive mode)
                    tracker.track_interactive(step_size=100)

                    # generate hypotheses and run the global optimizer
                    tracker.optimize()

                    ## export with guaranteed unique fn 
                    date = datetime.now().strftime("%Y_%m_%d_%I_%M_%S_%p")
                    tracker.export(os.path.join(root_folder, '{}/{}'.format(expt, pos), 'gfp_tracks_{}.h5'.format(date)), obj_type='obj_type_1')

                    # get the tracks in a format for napari visualization
                    #gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
                    #gfp_tracks = tracker.tracks

                ### track rfp objects 
                message = 'Tracking RFP objects'+ str(pos)
                log.append(message)
                print(message) 
                # initialise a tracker session using a context manager
                with btrack.BayesianTracker() as tracker:

                    # configure the tracker using a config file
                    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_scribble_sparse.json')
                    tracker.max_search_radius = 10

                    # append the objects to be tracked
                    tracker.append(rfp_obj_from_arr) 
                    #tracker.append(rfp_obj_from_generator)

                    # set the volume
                    tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

                    # track them (in interactive mode)
                    tracker.track_interactive(step_size=100)

                    # generate hypotheses and run the global optimizer
                    tracker.optimize()

                    ## export with guaranteed unique fn 
                    date = datetime.now().strftime("%Y_%m_%d_%I_%M_%S_%p")
                    tracker.export(os.path.join(root_folder, '{}/{}'.format(expt, pos), 'rfp_tracks_{}.h5'.format(date)), obj_type='obj_type_2')

                    # get the tracks in a format for napari visualization
                    #rfp_data, rfp_properties, rfp_graph = tracker.to_napari(ndim=2)
                    #rfp_tracks = tracker.tracks
                message = 'Finished'+ str(pos)
                log.append(message)
                print(message) 
                
        except Exception as err:
            message = str(pos)+ ' Failed to generate tracking file due to error:' + str(err)
            log.append(message)
            print(message) 

  0%|          | 0/24 [00:00<?, ?it/s]

Starting experimentMK0000
('Pos0', 'starting')
Finding mask filesPos0
Filtering mask filesPos0
Finding corresponding raw image filesPos0




  0%|          | 0/1014 [00:00<?, ?it/s]

  0%|          | 2/1014 [00:00<01:16, 13.29it/s]

Loading all imagesPos0




  0%|          | 4/1014 [00:00<01:05, 15.50it/s]

  1%|          | 6/1014 [00:00<01:16, 13.26it/s]

  1%|          | 8/1014 [00:00<01:11, 14.03it/s]

  1%|          | 10/1014 [00:00<01:09, 14.42it/s]

  1%|▏         | 13/1014 [00:00<00:57, 17.46it/s]

  1%|▏         | 15/1014 [00:00<00:55, 17.96it/s]

  2%|▏         | 18/1014 [00:01<00:55, 17.79it/s]

  2%|▏         | 21/1014 [00:01<00:50, 19.53it/s]

  2%|▏         | 23/1014 [00:01<00:52, 18.82it/s]

  2%|▏         | 25/1014 [00:01<00:53, 18.60it/s]

  3%|▎         | 27/1014 [00:01<00:52, 18.78it/s]

  3%|▎         | 30/1014 [00:01<00:50, 19.44it/s]

  3%|▎         | 33/1014 [00:01<00:47, 20.62it/s]

  4%|▎         | 36/1014 [00:01<00:48, 20.01it/s]

  4%|▍         | 39/1014 [00:02<00:51, 19.09it/s]

  4%|▍         | 41/1014 [00:02<00:50, 19.13it/s]

  4%|▍         | 43/1014 [00:02<00:52, 18.50it/s]

  4%|▍         | 45/1014 [00:02<00:53, 18.07it/s]

  5%|▍         | 47/1014 [00:02<00:54, 17.81it/s]

  5%|▍         | 49/1014 [00:02<

...

Tracking GFP objectsPos22


[INFO][2021/05/28 07:48:27 PM]  - Timing (Bayesian updates: 0.14ms, Linking: 0.07ms)
[INFO][2021/05/28 07:48:27 PM]  - Probabilities (Link: 0.99958, Lost: 1.00000)
[INFO][2021/05/28 07:48:27 PM]  - Stats (Active: 34, Lost: 2728, Conflicts resolved: 887)
[INFO][2021/05/28 07:48:27 PM] Tracking objects in frames 1000 to 1014 (of 1014)...
[INFO][2021/05/28 07:48:27 PM]  - Timing (Bayesian updates: 0.18ms, Linking: 0.08ms)
[INFO][2021/05/28 07:48:27 PM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2021/05/28 07:48:27 PM] SUCCESS.
[INFO][2021/05/28 07:48:27 PM]  - Found 674 tracks in 1014 frames (in 0.0s)
[INFO][2021/05/28 07:48:27 PM]  - Inserted 982 dummy objects to fill tracking gaps
[INFO][2021/05/28 07:48:27 PM] Loading hypothesis model: MDCK_hypothesis_Kristina
[INFO][2021/05/28 07:48:27 PM] Calculating hypotheses (relax: False)...
[INFO][2021/05/28 07:48:27 PM] Setting up constraints matrix for global optimisation...
[INFO][2021/05/28 07:48:27 PM] Optimizing...
[INFO][2021/

Tracking RFP objectsPos22


[INFO][2021/05/28 07:48:28 PM]  - Timing (Bayesian updates: 0.18ms, Linking: 0.08ms)
[INFO][2021/05/28 07:48:28 PM]  - Probabilities (Link: 0.99999, Lost: 0.90772)
[INFO][2021/05/28 07:48:28 PM]  - Stats (Active: 39, Lost: 6837, Conflicts resolved: 405)
[INFO][2021/05/28 07:48:28 PM] Tracking objects in frames 400 to 499 (of 800)...
[INFO][2021/05/28 07:48:28 PM]  - Timing (Bayesian updates: 0.16ms, Linking: 0.08ms)
[INFO][2021/05/28 07:48:28 PM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2021/05/28 07:48:28 PM]  - Stats (Active: 40, Lost: 7994, Conflicts resolved: 428)
[INFO][2021/05/28 07:48:28 PM] Tracking objects in frames 500 to 599 (of 800)...
[INFO][2021/05/28 07:48:28 PM]  - Timing (Bayesian updates: 0.14ms, Linking: 0.08ms)
[INFO][2021/05/28 07:48:28 PM]  - Probabilities (Link: 0.99999, Lost: 1.00000)
[INFO][2021/05/28 07:48:28 PM]  - Stats (Active: 36, Lost: 9629, Conflicts resolved: 481)
[INFO][2021/05/28 07:48:28 PM] Tracking objects in frames 600 to 699 (of 800

FinishedPos22
('Pos23', 'starting')
Finding mask filesPos23
No masks found forPos23, continuing to next position
Starting experimentMK0001
('Pos0', 'starting')
Finding mask filesPos0
No masks found forPos0, continuing to next position
('Pos1', 'starting')
Finding mask filesPos1
No masks found forPos1, continuing to next position
('Pos2', 'starting')
Finding mask filesPos2
No masks found forPos2, continuing to next position
('Pos3', 'starting')
Finding mask filesPos3
No masks found forPos3, continuing to next position
('Pos4', 'starting')
Finding mask filesPos4
No masks found forPos4, continuing to next position
('Pos5', 'starting')
Finding mask filesPos5
No masks found forPos5, continuing to next position
('Pos6', 'starting')
Finding mask filesPos6
No masks found forPos6, continuing to next position
('Pos7', 'starting')
Finding mask filesPos7
No masks found forPos7, continuing to next position
('Pos8', 'starting')
Finding mask filesPos8
No masks found forPos8, continuing to next positi

In [81]:
log

['Starting experimentMK0000',
 ('Pos0', 'starting'),
 'Finding mask filesPos0',
 'Filtering mask filesPos0',
 'Finding corresponding raw image filesPos0',
 'Loading all imagesPos0',
 'Localising objectsPos0',
 'Tracking iRFP objectsPos0',
 'Tracking GFP objectsPos0',
 'Tracking RFP objectsPos0',
 'FinishedPos0',
 ('Pos1', 'starting'),
 'Finding mask filesPos1',
 'No masks found forPos1, continuing to next position',
 ('Pos2', 'starting'),
 'Finding mask filesPos2',
 'No masks found forPos2, continuing to next position',
 ('Pos3', 'starting'),
 'Finding mask filesPos3',
 'No masks found forPos3, continuing to next position',
 ('Pos4', 'starting'),
 'Finding mask filesPos4',
 'No masks found forPos4, continuing to next position',
 ('Pos5', 'starting'),
 'Finding mask filesPos5',
 'Filtering mask filesPos5',
 'Finding corresponding raw image filesPos5',
 'Loading all imagesPos5',
 'Localising objectsPos5',
 'Tracking iRFP objectsPos5',
 'Tracking GFP objectsPos5',
 'Tracking RFP objectsPo